In [2]:
import glob
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim 
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering libraryimport glob

from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen


In [3]:
# Explore Suburbs

####### radius = 500 in US, increase radius in other countries ##########
def getNearbyVenues(names, latitudes, longitudes, radius=500,LIMIT = 100):

    # CLIENT_ID = 'TS2DPJEPPDPN5EZMGNMTHTJ25T3P0NOZM0D4LRFEHUUX1J1D' # your Foursquare ID
    # CLIENT_SECRET = 'VASOSLOQETUPKYRXOCPNJWQSXPCXKK12XD3KUY2FK5P2RKHE' # your Foursquare Secret
    # VERSION = '20180605' # Foursquare API version

    CLIENT_ID = 'GXU4GA3BAHCNZSSDABZFJ45HM4GUKVYJRNEIQ1E2KYCRBEXU'
    CLIENT_SECRET = 'ONMWLI5JTITCBZWYURJKMI532UBOXG4QMWKVNAMKXYOPRS4B'
    VERSION = '20180605'

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [4]:
def mapView(address, latitude_col, longitude_col, region_col, suburb_col, zoom_start):

    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))
    # create map of brisbane using latitude and longitude values
    map = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)
    # add markers to map
    for lat, lng, borough, neighborhood in zip(latitude_col, longitude_col, region_col, suburb_col):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map)

    return(map)


In [5]:
def clusterMap(address, vLatitude_col, vLongitude_col, address_col, clusterLabel_col, zoom_start):

    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(vLatitude_col, vLongitude_col, address_col, clusterLabel_col):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
        
    map_clusters
